Coursera capstone

In [2]:
zomatao_user_key = 'b348f057e5edb73209c2caa7f6dcb638'
kol_geojson = 'https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Kolkata/kolkata.geojson'

In [176]:
# Zomato web acesss
class Zomato:
    def __init__(self, user_key, content_type = 'application/json'):
        self._key = user_key
        self.request_count = 0
        self.content_type = content_type
        self._header = {
                    'Accept': content_type,
                    'user-key': user_key,
                            }
        self._categories_id = {}

    def get_response(self,api,params=''):
        import requests
        headers = self._header
        status = {  200 :'Sucessfully Connected to Zomato..',
                    403 :'Connection Error: Invalid User Keys',
                    404 : 'No Connection to Server..'
                }
        response = requests.get(api, headers=headers,params=params)
        print(status[response.status_code])
        if response.status_code == 200:
            self.request_count += 1
            return response.json()
    
    def categories(self,method='get'):
        api = 'https://developers.zomato.com/api/v2.1/categories'
        response = self.get_response(api)
        result_dict = {categories['categories']['name']: categories['categories']['id'] for categories in response['categories']}
        if method == 'set':
            self._categories_id = result_dict 
        return result_dict
    
    def cities(self,city_name='',id='',lat='',lon=''):
        params = (
            ('q', city_name),
            ('lat', lat),
            ('lon', lon),
            ('city_ids', id))

        api = 'https://developers.zomato.com/api/v2.1/cities'
        response = self.get_response(api,params=params)  
        return response

    def cuisines(self,city_id='',lat='',lon=''):
        params = (
            ('city_id', city_id),
            ('lat', lat),
            ('lon', lon))
        api = 'https://developers.zomato.com/api/v2.1/cuisines'
        response = self.get_response(api,params=params)  
        return response
    
    def geocode(self,lat='',lon=''):
        api = 'https://developers.zomato.com/api/v2.1/geocode'
        params = (
            ('lat', lat),
            ('lon', lon))
        response = self.get_response(api,params=params)  
        return response

    def location(self,city='',lat='',lon='',max_count=100):
        api = 'https://developers.zomato.com/api/v2.1/locations'
        params = (
        ('query', city),
        ('lat', lat),
        ('lon', lon),
        ('count', str(max_count)))
        response = self.get_response(api,params=params)  
        return response
    
    def location_details(self,city='',lat='',lon=''):
        r = self.location(city,lat,lon,1)
        entity_id = r['location_suggestions'][0]['entity_id']
        entity_type = r['location_suggestions'][0]['entity_type']

        params = (
            ('entity_id', entity_id),
            ('entity_type', entity_type))
        api = 'https://developers.zomato.com/api/v2.1/location_details'

        response = self.get_response(api,params=params)  
        return response
    
    def restaurant(self,res_id=''):
        api = 'https://developers.zomato.com/api/v2.1/restaurant'
        params = (
                ('res_id', res_id),)
        response = self.get_response(api,params=params) 
        return response
    
    def reviews(self,res_id='',start='1',count='10'):
        api = 'https://developers.zomato.com/api/v2.1/reviews'
        params = (
            ('res_id', res_id),
            ('start', start),
            ('count', count),) 
        response = self.get_response(api,params=params) 
        return response


In [177]:
zomato = Zomato(zomatao_user_key)

In [ ]:
# kolkata pincode corrdinates
from bs4 import BeautifulSoup #Web Scrapping
import pandas as pd #library for dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from tabulate import tabulate
import json #library for json file
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

with open("pincode.html", "r") as f:
    
    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')

pincodes = soup.find_all('div',class_='jbzYp')
pincodes = [pc.string for pc in pincodes]

cordinates = soup.find_all('div',class_='title')
cordinates = [(c.string.split(", ")[0][:7],c.string.split(", ")[1][:7]) for c in cordinates]

kolkata_pincodes = pd.DataFrame({'pincode':pincodes,'cordinates':cordinates}).sort_values(by='pincode')
#kolkata_pincodes.to_csv("kolkata.csv")

In [87]:
kolkata = pd.read_csv('kolkata_pincodes.csv',index_col=0)
kolkata.head()

,pincode,latitude,longitude
114,700122,22.766851,88.376411
113,700121,22.766614,88.422983
112,700120,22.752200,88.379000
111,700119,22.734883,88.396685
109,700117,22.723511,88.375425


In [89]:
address = 'Kolkata, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 22.54541245, 88.3567751581234.


In [126]:
map_kolkata = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, pincode in zip(kolkata['latitude'], kolkata['longitude'],kolkata['pincode']):
    label = '{}'.format(pincode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kolkata)  
    
map_kolkata

In [99]:
kolkata[kolkata['pincode'] == 700063]

,pincode,latitude,longitude
60,700063,22.464592,88.310792


In [103]:
pin_63 = zomato.geocode('22.464592','88.310792')


Sucessfully Connected to Zomato..


In [118]:
pin_63.keys()

dict_keys(['location', 'popularity', 'link', 'nearby_restaurants'])

In [155]:
import json
for res in pin_63['nearby_restaurants']:
    print()
    print(res['restaurant']['name']," : ", res['restaurant']['id'])
    print(res['restaurant']['location']['address'])
    print(res['restaurant']['cuisines'])
    print("{},{}".format(res['restaurant']['user_rating']['rating_text'],res['restaurant']['user_rating']['aggregate_rating']))
    


Domino's Pizza  :  23408
Riddhi Siddhi Garden, Near ESI Hospital, Joka, Kolkata
Pizza, Fast Food
Very Good,4.0

Benjamin Bangalee  :  18420484
P/122 James Long Sarani, Behala, Kolkata
Bengali, Chinese, Seafood, Continental, Momos
Good,3.8

Urban Kitchen  :  18240469
41, Diamond Harbour Road, Near Genexx Valley, Joka, Kolkata
North Indian, Chinese, Biryani
Good,3.8

Dubbawala  :  18309975
135/10 Diamond Harbour Road, Thakurpukur,
Bengali
Average,3.4

The Herbs & Spices  :  22424
Gennex Valley, Diamond Park, Opposite ESIC Hospital, Joka, Kolkata
North Indian, Chinese
Good,3.8

Flavours  :  18473784
90, James Long Sarani, Diamond Park, Joka, Kolkata
Chinese, North Indian, Rolls, Biryani, Mughlai
Good,3.9

Hungryy Birdz  :  18662812
A2/3, Srijani Park, Opposite Cancer Hospital, Thakur Pukur, Kolkata
Chinese, North Indian
Good,3.6

Hong Kong  :  18572855
308/1A, Diamond Harbour Road, Near Axis Bank, Thakurpukur Branch, Joka, Kolkata
Chinese, North Indian, Biryani, Momos
Good,3.6

Mio Amore

In [ ]:
dict_keys(['R', 'apikey', 'id', 'name', 'url', 'location', 'switch_to_order_menu', 'cuisines', 'average_cost_for_two', 'price_range', 'currency', 'offers', 'opentable_support', 'is_zomato_book_res', 'mezzo_provider', 'is_book_form_web_view', 'book_form_web_view_url', 'book_again_url', 'thumb', 'user_rating', 'photos_url', 'menu_url', 'featured_image', 'has_online_delivery', 'is_delivering_now', 'include_bogo_offers', 'deeplink', 'is_table_reservation_supported', 'has_table_booking', 'events_url'])

In [121]:
pin_63['location']['title']


'Behala'

In [122]:
pin_63['popularity']

{'popularity': '3.52',
 'nightlife_index': '0.00',
 'nearby_res': ['23408',
  '18420484',
  '18240469',
  '18309975',
  '22424',
  '18473784',
  '18662812',
  '18572855',
  '25351'],
 'top_cuisines': ['North Indian',
  'Chinese',
  'Biryani',
  'Fast Food',
  'Bengali'],
 'popularity_res': '100',
 'nightlife_res': '10',
 'subzone': 'Thakur Pukur',
 'subzone_id': 1093,
 'city': 'Kolkata'}

In [165]:
def geo(pincode,latitude,longitude):
    geo_dict = {
        'pincode':pincode, 'latitude': latitude, 'longitude': longitude
    }
    geo_result = zomato.geocode(latitude,longitude)
    geo_dict['location'] = geo_result['location']['title']
    geo_dict['subzone'] = geo_result['popularity']['subzone']
    geo_dict['top_cuisines'] = ','.join(geo_result['popularity']['top_cuisines'])
    geo_dict['popularity'] = geo_result['popularity']['popularity']
    geo_dict['nearby_res'] = ','.join(geo_result['popularity']['nearby_res'])

    return geo_dict

pd.DataFrame(geo('700063','22.464592','88.310792'),index=[0])
geo('700063','22.464592','88.310792')

Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..


{'pincode': '700063',
 'latitude': '22.464592',
 'longitude': '88.310792',
 'location': 'Behala',
 'subzone': 'Thakur Pukur',
 'top_cuisines': 'North Indian,Chinese,Biryani,Fast Food,Bengali',
 'popularity': '3.52',
 'nearby_res': '23408,18420484,18240469,18309975,22424,18473784,18662812,18572855,25351'}

In [166]:
zomato_kol = {}
for p,lag,lon,i in zip(kolkata['pincode'],kolkata['latitude'],kolkata['longitude'],kolkata.index):
    zomato_kol[i] = geo(p,lag,lon)

Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Co

In [171]:
kolkata_resturent = pd.DataFrame(zomato_kol).T
kolkata_resturent.head()

,pincode,latitude,longitude,location,subzone,top_cuisines,popularity,nearby_res
114,700122,22.7669,88.3764,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
113,700121,22.7666,88.423,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
112,700120,22.7522,88.379,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
111,700119,22.7349,88.3967,Khardah,Khardah,"Chinese,North Indian,Fast Food,Biryani,Bengali",3.95,"19127565,24311,25341,19114341,18198698,1860263..."
109,700117,22.7235,88.3754,Khardah,Khardah,"Chinese,North Indian,Fast Food,Biryani,Bengali",3.95,"19127565,24311,25341,19114341,18198698,1860263..."


In [172]:
map_kolkata = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, pincode,location in zip(kolkata_resturent['latitude'], kolkata_resturent['longitude'],kolkata_resturent['pincode'],kolkata_resturent['subzone']):
    label = '{}({})'.format(location,pincode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kolkata)  
    
map_kolkata

In [173]:
kolkata_resturent.to_csv('kolkata_resturent.csv')

In [178]:
zomato.restaurant('19127565')

Sucessfully Connected to Zomato..


{'R': {'has_menu_status': {'delivery': -1, 'takeaway': -1},
  'res_id': 19127565},
 'apikey': 'b348f057e5edb73209c2caa7f6dcb638',
 'id': '19127565',
 'name': 'Dada Boudi Restaurant',
 'url': 'https://www.zomato.com/kolkata/dada-boudi-restaurant-1-sodepur?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
 'location': {'address': '8, BT Road, Thakur Corner, Sodepur, Kolkata',
  'locality': 'Sodepur',
  'city': 'Kolkata',
  'city_id': 2,
  'latitude': '22.7023589164',
  'longitude': '88.3748411760',
  'zipcode': '',
  'country_id': 1,
  'locality_verbose': 'Sodepur, Kolkata'},
 'switch_to_order_menu': 0,
 'cuisines': 'North Indian, Chinese, Mughlai',
 'timings': '12noon – 10:30pm (Mon-Sun)',
 'average_cost_for_two': 700,
 'price_range': 2,
 'currency': 'Rs.',
 'highlights': ['Dinner',
  'Delivery',
  'Lunch',
  'Cash',
  'Takeaway Available',
  'Indoor Seating'],
 'offers': [],
 'opentable_support': 0,
 'is_zomato_book_res': 0,
 'mezzo_provider': 'OTHER',
 'is_book_form_web_vie

In [192]:
review = zomato.reviews('19127565')['user_reviews']

Sucessfully Connected to Zomato..


In [195]:
for r in review:
    print(r['review']['rating'])
    print(r['review']['review_text'])

4
📸MUTTON BIRIYANI (Highly Recommended):the Biriyani was aromatic and flavourful.Both the mutton piece and the potato compared to other places.the mutton was nicely cooked,juicy,and tender.the potato was soft and flavourful.and the quantity of the rice was also great.over all it is worth the money.this is one of the best places to have biriyani. 📸 *CHICKEN TIKKA BUTTER MASALLA* : It's the best companion, be it with biriyani or naan. The tangy buttery creamy thick gravy creates a sweet yet spicy sensation on your tastebuds. . . 📌 DADA BOUDI RESTURANT, Sodpur, Kolkata 🌟 Overall Rating: 4.5/5 🌟  . . . For more such delicious posts follow us on Instagram, Facebook and Zomato @_bitesnburps_ . . .
1

5

3

5
I had Dada Boudir biriyani sometime back ..I had to review this since I just loved it.. perhaps it is one of the best biriyanis i had..everything was so perfect.. can't wait to have it again!!🙂🙂
4
Barrackpure means dada boudi's Biriyani. Now they have opened a restaurant at Sodpur. Be it

In [194]:
review[0]['review']

{'rating': 4,
 'review_text': "📸MUTTON BIRIYANI (Highly Recommended):the Biriyani was aromatic and flavourful.Both the mutton piece and the potato compared to other places.the mutton was nicely cooked,juicy,and tender.the potato was soft and flavourful.and the quantity of the rice was also great.over all it is worth the money.this is one of the best places to have biriyani. 📸 *CHICKEN TIKKA BUTTER MASALLA* : It's the best companion, be it with biriyani or naan. The tangy buttery creamy thick gravy creates a sweet yet spicy sensation on your tastebuds. . . 📌 DADA BOUDI RESTURANT, Sodpur, Kolkata 🌟 Overall Rating: 4.5/5 🌟  . . . For more such delicious posts follow us on Instagram, Facebook and Zomato @_bitesnburps_ . . .",
 'id': 48764775,
 'rating_color': '5BA829',
 'review_time_friendly': 'yesterday',
 'rating_text': 'Great!',
 'timestamp': 1584266727,
 'likes': 1,
 'user': {'name': 'Akash Sarkar',
  'foodie_color': 'ffae4f',
  'profile_url': 'https://www.zomato.com/users/akash-sarkar

In [196]:
zomato.request_count

6